In [2]:
from analyse_fit_files.parse_fit_file import get_fit_file_data, sport_peak_curve
from analyse_fit_files.visualize_fit_files import plot_sport_peak_curve

ImportError: cannot import name 'sport_peak_curve' from 'analyse_fit_files.parse_fit_file' (/Users/kevin/repos/analyse-fit-files/env/src/analyse-fit-files/analyse_fit_files/parse_fit_file.py)

In [ ]:
ride = get_fit_file_data(
    "/Users/kevin/repos/analyse-fit-files/data/Zwift_-_Going_Coastal_in_Watopia.fit"
)
cycling = sport_peak_curve(fit_file_dataframe=ride, sport="cycling")

In [ ]:
pace = get_fit_file_data(
    "/Users/kevin/repos/analyse-fit-files/data/Morning_Run_2024_04_04.fit"
)
speed = sport_peak_curve(fit_file_dataframe=pace, sport="running")

secondary = False
if secondary:
    pace1 = get_fit_file_data(
        "/Users/kevin/repos/analyse-fit-files/data/Morning_Run_2024_01_25.fit"
    )
    speed1 = sport_peak_curve(fit_file_dataframe=pace1, sport="running")

In [ ]:
plot_sport_peak_curve(primary_fitfile_dataframe=ride, primary_sport_peak_curve=cycling, secondary_fitfile_dataframe=None, secondary_sport_peak_curve=None, sport="cycling")